# 5.6.1 Affine 계층

신경망의 순전파에서는 가중치 신호의 총합을 계산하기 때문에 행렬의 곱을 사용했다. 입력에 가중치를 곱하고 편향을 더한 값을 활성화 함수로 변환하고 다음 층으로 전파하는 것이 신경망 순전파의 흐름이었다. 행렬의 곱 계산은 대응하는 차원의 원소 수를 일치시키는 것이 핵심이다. 예를 들어, 입력과 가중치의 곱은 밑의 그림과 같이 대응하는 차원의 원소 수를 일치시켜야 한다. 

<img src=images/5_23.png height=100px width=300px>

그럼 앞에서 수행한 계산을 계산 그래프로 그려보자. 곱을 계산하는 노드를 "dot"라고 하면, X(입력)@W(가중치) + B(편향) 계산은 다음과 같이 그려진다. 또한, 각 변수의 이름 위에 그 변수의 형상도 표기한다.

<img src=images/5_24.png height=100px width=500px>

지금까지의 계산 그래프와는 다르게 이 계산 그래프에는 행렬이 흐르고 있다.

이제 위의 그림의 역전파를 생각해보자. 행렬을 사용한 역전파도 행렬의 원소마다 전개해보면 스칼라값을 사용한 지금까지의 계산 그래프와 같은 순서로 생각할 수 있다. 실제로 전개해보면 다음 식이 도출 된다.

$
{\partial L \over \partial X} =
{\partial L \over \partial Y} \cdot {W^T}
\qquad \\
{\partial L \over \partial W} = 
{X^T} \cdot {\partial L \over \partial Y}
\qquad$ [식 5.13]

<img src=images/5_25_1.jpg height=100px width=500px>

자세한 계산은 다음과 같다. ${\partial Y \over \partial W}$는 모르겠다.

따라서 최종적인 역전파는 다음과 같다.

<img src=images/5_25.png height=100px width=500px>

계산 그래프에서는 각 변수의 형상에 주의해야 한다. 특히, __X__ 와 ${\partial L \over \partial X}$는 같은 형상을 가지고, __W__ 와 ${\partial L \over \partial W}$도 같은 형상을 가진다. __X__ 와 ${\partial L \over \partial X}$가 같은 형상을 가진다는 것은 아래의 식을 보면 명확해진다.

$
X = (x_0, x_1,\cdots,x_n) \\
{\partial L \over \partial X} = 
\begin{pmatrix}
{\partial L \over \partial x_0}, &
{\partial L \over \partial x_1}, &
\cdots,&
{\partial L \over \partial x_n}
\end{pmatrix}
\qquad$ [식 5.15]

왜 형렬의 형상에 주의해야 할까? 행렬의 곱에서는 대응하는 차원의 원소 수를 일치시켜야 하는데, 이를 위해서는 [식 5.13]을 동원해야 할 수도 있기 때문이다. 예를 들어 ${\partial L \over \partial Y}$의 형상이 (3,)이고, W의 형상이 (2,3)일 때, ${\partial L \over \partial X}$의 형상이 (2,)가 되는 ${\partial L \over \partial Y}$ 과 W의 곱을 생각해보자. 그러면 자연히 [식 5.13]이 유도될것이다.

<img src=images/5_26.png height=100px width=500px>

# 5.6.2 배치용 Affine 계층

지금까지 설명한 Affine 계층은 입력 데이터로 __X__ 하나만을 고려한 것이었다. 이번 절에서는 데이터 N개를 묶어 순전파하는 경우, 즉 배치용 Affine 계층을 생각해보자.<br>
배치용 Affine 계층을 계산 그래프로 그려보면 다음과 같다.

<img src=images/5_27.png height=100px width=500px>

기존과 다른 부분은 입력인 X의 형상이 (N,2)가 된 것 뿐이다. 그 뒤로는 지금까지와 같이 계산 그래프의 순서를 따라 순순히 행렬 계산을 하게 된다. 또, 역전파 때는 행렬의 형상에 주의하면 ${\partial L \over \partial X}과 {\partial L \over \partial W}$는 이전과 같이 도출할 수 있다.

편향을 더할 때도 주의해야 한다. 데이터가 여러개로 늘어나면 편향은 각 데이터에 대해 더해지게 되어 이후 역전파를 계산할 때 각 데이터의 역전파 값이 편향의 원소에 모여야한다. 편향의 변화가 모든 데이터들의 변화를 야기하기 때문이다.

이상의 Affine 구현은 다음과 같다. Affine 구현은 입력 데이터가 텐서(4차원 데이터)인 경우도 고려한 것이라 다음 구현과는 약간 차이가 있다.

In [2]:
class AffineLayer:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = x @ self.W + self.b
        
        return out
    
    def backward(self, dout):
        dx = dout @ self.W.T
        self.dW = self.x.T @ dout
        self.db = np.sum(dout, axis=0)
        
        return dx

# 5.6.3 Softmax-with-Loss 계층

마지막으로 출력층에서 사용하는 소프트맥스 함수에 관해 설명하겠다. 앞에서 말했듯이 소프트맥스 함수는 입력 값을 정규화하여 출력한다. 예를 들어 손글씨 숫자 인식에서의 Softmax 계층은 다음과 같다.

<img src=images/5_28.png height=100px width=500px>

다음과 같이 Softmax 계층은 입력 값을 정규화하여 출력한다. 또한, 손글씨 숫자는 가짓수가 10개이므로 Softmax 게층의 입력은 10개가 된다.

#### NOTE
신경망에서 수행하는 작업은 __학습__ 과 __추론__ 두 가지이다. 추론할 때는 일반적으로 Softmax 계층을 사용하지 않는다. 예컨대, 위의 그림의 신경망은 추론할 때는 마지막 Affine 계층의 출력을 인식 결과로 이용한다. 또한, 신경망에서 정규화하지 않는 출력 결과를 __점수(score)__ 라고 한다. 즉, 신경망 추론에서 답을 하나만 내는 경우에는 가장 높은 점수만 알면 되므로 Softmax 계층은 필요가 없다. 그러나 학습을 할 때에는 Softmax 계층이 필요하다. 학습을 할 시에는 손실함수를 필요로 하기 때문이다.

이제 소프트맥스 계층을 구현할 텐데, 손실 함수인 교차 엔트로피 오차도 포함하여 "Softmax_with_Loss 계층"이라는 이름으로 구현하자. 그 전에 먼저 Softmax_with_Loss 계층의 계산 그래프를 살펴보자.

<img src=images/5_29.png height=300px width=700px>

보다시피 Softmax_with_Loss 계층은 다소 복잡하다. 밑에 간소화한 그림을 그려놓았다.

<img src=images/5_30.png height=100px width=500px>

이 계산 그래프를 만드는 과정은 "__Appendix_A.ipynb__"파일에 담겨있다.

위 그림에서 소프트맥스 함수는 "Softmax" 계층으로, 교차 엔트로피 오차는 "Cross Entropy Error" 계층으로 표기했다. 여기에서는 3가지 클래스 분류를 가정하고 이전 계층에서 3개의 입력(점수)을 받는다.그림과 같이 Softmax 계층은 입력 $(a_1, a_2, a_3)$를 정규화하여 $(y_1, y_2, y_3)$를 출력한다. Cross Entropy Error 계층은 Softmax의 출력 $(y_1, y_2, y_3)$와 정답 레이블 $(t_1, t_2, t_3)$를 받고, 이 데이터들로부터 손실 L을 출력한다.<br>
역전파의 결과는 $(y_1 - t_1, y_2 - t_2, y_3 - t_3)$으로 "말끔한" 결과를 내놓고 있다. 신경망의 역전파에서는 이 차이인 오차가 앞 계층에 전해지는 것이다. 이는 신경망 학습의 중요한 성질이다.<br>
그런데 신경망 학습의 목적은 신경망의 출력(Softmax의 출력)이 정답 레이블과 가까워지도록 가중치 매개변수의 값을 조정하는 것이다. 그래서 신경망의 출력과 정답 레이블의 오차를 효율적으로 앞 계층에 전달해야 한다. 앞의 $(y_1 - t_1, y_2 - t_2, y_3 - t_3)$라는 결과는 바로 Softmax 계층의 출력과 정답 레이블의 차이로, 신경망의 현재 출력과 정답 레이블의 오차를 있는 그대로 드러내는 것이다.<br>
예를 들자면, 가령 정답 레이블이 (0,1,0)일 때, Softmax 계층이 (0.3, 0.2, 0.5)를 출력했다고 하자. 정답의 인덱스는 1로 확률로는 0.2(20%)라서 , 이 시점에서 신경망은 제대로 인식하지 못하고 있다. 이 결우 Softmax 계층의 역전파는 (0.3, -0.8, 0.5)라는 커다란 오차를 전파한다. 결과적으로 Softmax 계층의 앞 계층들은 그 큰 오차로부터 큰 깨달음을 얻게 된다. 반대의 경우로 같은 정답 레이블에 대해 Softmax 계층이 (0.01, 0.99, 0)을 출력한 경우라면 Softmax 계층이 전파하는 역전파는 (0.01, -0.01, 0)으로 비교적 작은 값을 보내고 학습하는 양도 줄어들게 된다.

In [1]:
from functions  import Function as F

class SoftmaxLossLayer:
    def __init__(self):
        self.Loss = None ## 손실
        self.t = None    ## 정답 레이블
        self.out = None  ## softmax의 출력
        
    def forward(self , x, t):
        self.t = t        
        self.out = F.softmax(x)
        self.Loss = F.cross_entropy_error(self.out, self.t)
        
        return self.Loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = dout * (self.out - self.t) / batch_size
        ## batch_size로 나누는 이유는 "/"노드에서 갈라진 값들이 모이게 된다. 하나의 데이터에 대해 역전파의
        ## 합은 (역전파) * 1이 되므로 batch들이 한데 모이게 되는 노드에서는 (역전파) * batch_size가
        ## 된다. 따라서 하나의 데이터의 영향을 알고 싶다면, 즉 dx는 (역전파) / batch_size가 되는 것이다.
        
        return dx